## Базовий промпт
Створимо базовий промпт для оптимізації паралельних обчислень на CUDA.
Визначимо характерні риси які має мати наш промт:
1. Чітке формулювання задачі: аби модель могла коректно зрозуміти наші вимоги.
2. Використання ролі експерта: запит до GPT як до фахівця з CUDA аби надати йому роль в якій він буде давати відповідь.
3. Конкретні вимоги: оптимізація пам’яті, синхронізація і т.д.
4. Формат виводу: C++ з коментарями та поясненнями аби одразу отримати готовий результат в якому можна буде розібратись.

Результат:

"You are an expert in GPU parallel computing and CUDA optimization. Your task is to generate an optimized CUDA kernel and corresponding host-side code for efficient parallel execution. The goal is to minimize execution time while ensuring correct synchronization and memory access patterns. The solution should include:
An optimized CUDA kernel that efficiently utilizes shared memory, registers, and global memory access patterns.
Proper thread block and grid size configuration for maximum performance.
Techniques to avoid memory access bottlenecks, such as coalesced memory access and minimizing bank conflicts.
Synchronization strategies to prevent race conditions and ensure correct results.
A sample workload and benchmarking methodology to compare performance gains.
Provide the implementation in C++ with CUDA and include explanations of key optimization choices. Format the response as well-structured code with comments and an explanation of the optimizations used."

In [3]:
import os
from openai import OpenAI
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("Please set OPENAI_API_KEY environment variable in your GitHub Codespace secrets.")

client = OpenAI(api_key=api_key)
deployment="gpt-3.5-turbo"

def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0,
        max_tokens=1024
    )
    return response.choices[0].message.content

text = f"""
You are an expert in GPU parallel computing and CUDA optimization. Your task is to generate an optimized CUDA kernel and corresponding host-side code for efficient parallel execution. The goal is to minimize execution time while ensuring correct synchronization and memory access patterns. The solution should include:
An optimized CUDA kernel that efficiently utilizes shared memory, registers, and global memory access patterns.
Proper thread block and grid size configuration for maximum performance.
Techniques to avoid memory access bottlenecks, such as coalesced memory access and minimizing bank conflicts.
Synchronization strategies to prevent race conditions and ensure correct results.
A sample workload and benchmarking methodology to compare performance gains.
Provide the implementation in C++ with CUDA and include explanations of key optimization choices. Format the response as well-structured code with comments and an explanation of the optimizations used.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)


```cpp
#include <iostream>
#include <cuda_runtime.h>

#define N 1024 // Number of elements in the array
#define BLOCK_SIZE 256 // Number of threads per block

// CUDA kernel to perform element-wise addition of two arrays
__global__ void vectorAddition(int* a, int* b, int* c)
{
    __shared__ int sharedMem[BLOCK_SIZE]; // Shared memory for each block
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    // Load data into shared memory
    sharedMem[tid] = a[i] + b[i];
    __syncthreads(); // Ensure all threads have finished loading data

    // Perform addition using shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1)
    {
        if (tid < s)
        {
            sharedMem[tid] += sharedMem[tid + s];
        }
        __syncthreads(); // Ensure all threads have finished current iteration
    }

    // Write result back to global memory
    if (tid == 0)
    {
        c[blockIdx.x] = sharedMem[0];
    }
}

int main()
{
    int *a, *b, *c; // Hos

## Проведення токенізації
Візьмемо вже відомий нам код з першої комірки виконання базових вправ і проаналізуємо наш промпт:

In [4]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
You are an expert in GPU parallel computing and CUDA optimization. Your task is to generate an optimized CUDA kernel and corresponding host-side code for efficient parallel execution. The goal is to minimize execution time while ensuring correct synchronization and memory access patterns. The solution should include:
An optimized CUDA kernel that efficiently utilizes shared memory, registers, and global memory access patterns.
Proper thread block and grid size configuration for maximum performance.
Techniques to avoid memory access bottlenecks, such as coalesced memory access and minimizing bank conflicts.
Synchronization strategies to prevent race conditions and ensure correct results.
A sample workload and benchmarking methodology to compare performance gains.
Provide the implementation in C++ with CUDA and include explanations of key optimization choices. Format the response as well-structured code with comments and an explanation of the optimizations used.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);
print(f"Number of tokens: {len(tokens)}")

# Decode the integers to see what the text versions look like
decoded_tokens = [encoding.decode_single_token_bytes(token) for token in tokens]
print(decoded_tokens)

# Calculate the average length of tokens
total_length = sum(len(token) for token in decoded_tokens)
average_length = total_length / len(decoded_tokens) if len(decoded_tokens) > 0 else 0

# Print the average length of tokens
print(f"Average length of tokens: {average_length:.2f}")

[198, 2675, 527, 459, 6335, 304, 23501, 15638, 25213, 323, 55909, 26329, 13, 4718, 3465, 374, 311, 7068, 459, 34440, 55909, 10206, 323, 12435, 3552, 25034, 2082, 369, 11297, 15638, 11572, 13, 578, 5915, 374, 311, 30437, 11572, 892, 1418, 23391, 4495, 59012, 323, 5044, 2680, 12912, 13, 578, 6425, 1288, 2997, 512, 2127, 34440, 55909, 10206, 430, 30820, 60880, 6222, 5044, 11, 25771, 11, 323, 3728, 5044, 2680, 12912, 627, 1360, 716, 4617, 2565, 323, 5950, 1404, 6683, 369, 7340, 5178, 627, 29356, 8467, 311, 5766, 5044, 2680, 11176, 35291, 14895, 11, 1778, 439, 1080, 3916, 2041, 5044, 2680, 323, 77391, 6201, 26885, 627, 50, 69029, 15174, 311, 5471, 7102, 4787, 323, 6106, 4495, 3135, 627, 32, 6205, 54696, 323, 29531, 287, 38152, 311, 9616, 5178, 20192, 627, 61524, 279, 8292, 304, 356, 1044, 449, 55909, 323, 2997, 41941, 315, 1401, 26329, 11709, 13, 15392, 279, 2077, 439, 1664, 12, 52243, 2082, 449, 6170, 323, 459, 16540, 315, 279, 82278, 1511, 627]
Number of tokens: 163
[b'\n', b'You', b' are

## Оцінка якості результатів

Оптимальне використання shared memory, правильна синхронізація, ефективний розмір блоків, коректне керування пам’яттю. Відповідь дана на мові C++, коментарі присутні, в цілому наш промт був повністю виконаний. Що до оптимальності результатів роботи самого коду можна було б дати відповідь тільки самому будучи експертом.



## Запропонувати покращення

На основі отриманих результатів та аналізу токенізації, ми можемо запропонувати наступні покращення для оптимізації паралельних обчислень на CUDA:
- Використання більш специфічних інструкцій для моделі.
- Додавання додаткових деталей про архітектуру CUDA та можливості паралельних обчислень.
- Розробка більш складних промптів для врахування різних сценаріїв паралельних обчислень.